https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`



In [1]:
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama import OllamaEmbeddings

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding= OllamaEmbeddings(
        model="nomic-embed-text:v1.5"
    ),
)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Try to write it yourself

Grader:

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel
from typing import Literal



retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
print(retrieval_grader.invoke({"question": question, "document": docs[1].page_content}))

/home/zhiyong/.pyenv/versions/3.10.4/envs/rag/lib/python3.10/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value typing.Literal['Yes', 'No'] is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


content='{ "binary_score": "yes" }' additional_kwargs={} response_metadata={'model': 'llama3.2:3b-instruct-q8_0', 'created_at': '2025-02-17T15:03:48.357281973Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13088469335, 'load_duration': 5657294925, 'prompt_eval_count': 279, 'prompt_eval_duration': 5816000000, 'eval_count': 10, 'eval_duration': 836000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-6363d100-469e-4d38-b389-81d9a7b6623d-0' usage_metadata={'input_tokens': 279, 'output_tokens': 10, 'total_tokens': 289}


In [ ]:
# Generate
generation_prompt_template = """You are a RAG system tasked with answering a user query given a few documents for reference.

{documents}

user query:
{user_query}"""
grade_prompt = ChatPromptTemplate.from_template(generation_prompt_template)

generation_and_rewrite_llm = ChatOllama(
    model="llama3.2:3b-instruct-q8_0",
    temperature=0,
)

I don't really wanna set up Tavilysearch. Will hold off for now.

Define graph state

In [11]:
from langchain.schema import Document

In [10]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
    question: str
    generation: str
    web_search: str
    documents: List[str]

1. retrieve
2. grade
   1. rewrite -> web search
3. generation

In [ ]:
# Key point to note is that we always manipulate state, reminds me of ansible and idempotence
def retrieve(state):
    # Note that retriever is a global obj
    user_query = state['user_query']
    return {
        'user_query': user_query, 
        'docs': retriever.get_relevant_documents(user_query)
    }

def grading(state):
    # Data model
    class GradeDocuments(BaseModel):
        """Binary score for relevance check on retrieved documents."""
        binary_score: str = Literal['Yes', "No"]

    structured_llm_grader = ChatOllama(
        model="llama3.2:3b-instruct-q8_0",
        temperature=0,
        format=GradeDocuments.model_json_schema())

    # Prompt
    grade_prompt_template = """You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.

question:
{user_query}

document:
{document}
"""
    grade_prompt = ChatPromptTemplate.from_template(grade_prompt_template)


    user_query = state['user_query']
    docs = state['docs']
    filtered_docs = []
    search = False

    for i, doc in enumerate(docs):
        relevant = (grade_prompt | structured_llm_grader | StrOutputParser()).invoke({
            'user_query': user_query,
            'document': doc.page_content.strip()
        })
        if relevant == 'Yes':
            filtered_docs.append(doc)
            print(f'Doc {i+1} is relevant.')
        else:
            search = True
            print(f'Doc {i+1} is not relevant. Search will be done.')
    return {
        'user_query': user_query, 
        'docs': docs,
        'filtered_docs': filtered_docs,
        'search': search
    }

def rewrite(state):
    user_query = state['user_query']
    docs = state['docs']
    filtered_docs = state['filtered_docs']
    search = state['search']

    # rewrite question for the purposes of search
    rewrite_template = """You are a master of search engines and are tasked to rewrite a user query for optimum web search.

    user query:
    {user_query}"""
    rewrite_prompt = ChatPromptTemplate.from_template(rewrite_template)

    llm = ChatOllama(
        model="llama3.2:3b-instruct-q8_0",
        temperature=0,
    )
    rewritten_question = (rewrite_prompt | llm | StrOutputParser()).invoke({'user_query': user_query})
    return {
        'user_query': user_query, 
        'docs': docs,
        'filtered_docs': filtered_docs,
        'search': search,
        'rewritten_question': rewritten_question
    }



